In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import nx_cugraph as nxcg
from time import time
import os

print(f"using networkx version {nx.__version__}")

G = nx.read_graphml("binet-AMZ-Rw.graphml")
print("|N|",G.number_of_nodes(), "   |M|",G.number_of_edges())

nxcg_G = nxcg.from_networkx(G) 

a = time()
#bc_results = nx.betweenness_centrality(G, k=100)
nx.betweenness_centrality(nxcg_G, k=1000)
b = time() - a
print("GPU - time: %.10f seconds." % b)

a = time()
bc_results = nx.betweenness_centrality(G, k=1000)
#nx.betweenness_centrality(nxcg_G, k=1000)
b = time() - a
print("CPU - time: %.10f seconds." % b)

/home/daniel/miniconda3/envs/tesis/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


using networkx version 3.2
|N| 17079    |M| 32769
GPU - time: 3.4237275124 seconds.
CPU - time: 30.1187062263 seconds.
